训练集=测试集，会不会比随即好？

In [1]:
# import agent
# import environment
# import runner
import graph
import bus_line
import numpy as np
import networkx as nx
import sys
import random
import pylab 
import matplotlib.pyplot as plt
import math

In [2]:
import random

## polulation density：
$$ K(r) = K_0 e^{-b*r} $$
where in Beijing on 2010, K_0 = 36000 and b = 0.01

In [3]:
def polulation_density_from_center(r):
    return 36000*np.exp(-0.01*r)

center_node = [ i+40 for i in range(0,100) ]
center_pos = np.reshape([[ (i*5,j*5) for j in range(10) ] for i in range(10)],(100,2))

center_polulation = []
for i in range(len(center_node)):
    r = np.linalg.norm(center_pos[i]-np.array([22.5,22.5]))
    center_polulation.append( polulation_density_from_center(r) )

In [4]:
def dis_from_center(x):
    return x/8 + 3
#
def findAngle( x1, y1, x2, y2):
    angle = 0.0
    dx = x2 - x1
    dy = y2 - y1
    if x2 == x1:
        if y2 == y1 :
            angle = 0.0
        elif y2 < y1 :
            angle = 3.0 * math.pi / 2.0
        else:
            angle = math.pi / 2.0
    elif y1 == y2:
        if x2 == x1 :
            angle = 0.0
        elif x2 < x1 :
            angle =  math.pi 
        else:
            angle = 0.0
    elif x2 > x1 and y2 > y1: #1象限
        angle = math.atan(dx / dy)
    elif x2 > x1 and y2 < y1 :#4象限
        angle = 2.0 * math.pi - math.atan(-dy / dx)
    elif x2 < x1 and y2 < y1 :#3象限
        angle = math.pi + math.atan( dy / dx)
    elif x2 < x1 and y2 > y1 :#2象限
        angle = math.pi - math.atan(dy / -dx)
    return angle

def create_bus_line( start_bus_stop,  nb_bus_stops):
    bus_line = [start_bus_stop]
    angle_initial = findAngle(start_bus_stop[0],start_bus_stop[1],22.5,22.5)
    angle_0 = np.random.normal( angle_initial,math.pi/8 )
    for i in range(nb_bus_stops-1):
        if i == 0:
            old_bus_stop = bus_line[-1]
            distance = dis_from_center( np.linalg.norm(old_bus_stop-np.array([22.5,22.5])) )
            new_bus_stop = old_bus_stop+np.array( [distance*math.cos(angle_0),distance*math.sin(angle_0)] )
            bus_line.append( new_bus_stop )
        else:
            angle_0 = np.random.normal( angle_0,math.pi/8 )
            old_bus_stop = bus_line[-1]     
            distance = dis_from_center( np.linalg.norm(old_bus_stop-np.array([22.5,22.5])) )
            new_bus_stop = old_bus_stop+np.array( [distance*math.cos(angle_0),distance*math.sin(angle_0)] )
            bus_line.append( new_bus_stop )
    return np.array( bus_line )

def is_correct_line(line):
    for i in range(len(line)):
        if line[i][0]<0 or line[i][0]>45 or line[i][1]<0 or line[i][1]>45:
            return False
    return True

## 1 CREATE graph_dic

In [5]:
# graph_dic = {}

# for graph_ in range(200):
#     line_list = []
#     while len(line_list) < 8:
#         line = create_bus_line( np.array( [random.uniform(0,45),random.uniform(0,45)] ), 10 )
#         if is_correct_line(line):
#             line_list.append( line )

#     line_1 = line_list[0]
#     line_2 = line_list[1]
#     line_3 = line_list[2]
#     line_4 = line_list[3]
#     line_5 = line_list[4]
#     line_6 = line_list[5]
#     line_7 = line_list[6]
#     line_8 = line_list[7]
#     #create bus_line
#     bus_line_1 = bus_line.Bus_line( 1,1, [i for i in range(10)],     dict( zip([i for i in range(10)],     line_1.tolist())) )
#     bus_line_2 = bus_line.Bus_line( 2,1, [i for i in range(10,20)],dict( zip([i for i in range(10,20)], line_2.tolist())) )
#     bus_line_3 = bus_line.Bus_line( 3,1, [i for i in range(20,30)],dict( zip([i for i in range(20,30)], line_3.tolist())) )
#     bus_line_4 = bus_line.Bus_line( 4,1, [i for i in range(30,40)],dict( zip([i for i in range(30,40)], line_4.tolist())) )
#     bus_line_5 = bus_line.Bus_line( 5,1, [i for i in range(40,50)],dict( zip([i for i in range(40,50)], line_5.tolist())) )
#     bus_line_6 = bus_line.Bus_line( 6,1, [i for i in range(50,60)],dict( zip([i for i in range(50,60)], line_6.tolist())) )
#     bus_line_7 = bus_line.Bus_line( 7,1, [i for i in range(60,70)],dict( zip([i for i in range(60,70)], line_7.tolist())) )
#     bus_line_8 = bus_line.Bus_line( 8,1, [i for i in range(70,80)],dict( zip([i for i in range(70,80)], line_8.tolist())) )

    
    
#     line_all = np.vstack((line_1,line_2,line_3,line_4,line_5,line_6,line_7,line_8))

#     list_connection = []
#     for i in range( np.shape( line_all )[0]):
#         for j in range(i,np.shape( line_all )[0]):
#             if j != i and np.linalg.norm( line_all[i]-line_all[j] ) < 3:
#                 list_connection.append( [i,j])
    
#     list_waiting_time = []
#     list_waiting_time.append( bus_line_1.waiting_time ) 
#     list_waiting_time.append( bus_line_2.waiting_time )
#     list_waiting_time.append( bus_line_3.waiting_time )
#     list_waiting_time.append( bus_line_4.waiting_time )
#     list_waiting_time.append( bus_line_5.waiting_time )          
#     list_waiting_time.append( bus_line_6.waiting_time )     
#     list_waiting_time.append( bus_line_7.waiting_time )  
#     list_waiting_time.append( bus_line_8.waiting_time )  
#     #create graph
#     g  = graph.Graph( list_connection, list_waiting_time )
#     #add each bus_line
#     g.add_bus_line(bus_line_1)
#     g.add_bus_line(bus_line_2)
#     g.add_bus_line(bus_line_3)
#     g.add_bus_line(bus_line_4)
#     g.add_bus_line(bus_line_5)
#     g.add_bus_line(bus_line_6)    
#     g.add_bus_line(bus_line_7)  
#     g.add_bus_line(bus_line_8)  
#     #add connection between bus_line 
#     for connection in list_connection:
#         travel_time = np.linalg.norm(np.array( g.all_pos[connection[0]] )-np.array( g.all_pos[connection[1]] ))*100/300
#         g.add_connection( [ ( connection[0],connection[1],travel_time+list_waiting_time[connection[1]//10 ])] ) 
#         g.add_connection([(connection[1],connection[0],travel_time+list_waiting_time[connection[0]//10])]) 

#     graph_dic[graph_]= g

# np.save( 'test_set.npy',graph_dic )

In [6]:
# graph_dic = np.load( 'train_set.npy',allow_pickle=True ).item()

In [7]:
# graph_dic1 = np.load( 'test_set.npy',allow_pickle=True ).item()

In [8]:
# graph_dic_0 = np.load( 'train_set.npy',allow_pickle=True ).item()
# graph_dic_train = {}

# train_size = 7500
# before = 7250
# #每250保存一个模型
# for i in range(before,9000):
#     if i < train_size :
#         graph_dic_train[i-before] = graph_dic1[i]

In [9]:
graph_dic = np.load( 'test_set.npy',allow_pickle=True ).item()
# graph_dic = np.load( 'test_set_1.npy',allow_pickle=True ).item()
graph_dic_train = {}
graph_dic_test = {}

number_graphs = 50 #50
number_trees = 20


for i in range(number_graphs):
    for j in range(number_trees):
        graph_dic_test[i*number_trees+j] = graph_dic[i]



# for i in range(50):
#     graph_dic_test[i] = graph_dic[3]

# for i in range(200):
# #     j = np.random.randint(10)
#     if i<100:
#         graph_dic_train[i] = graph_dic[i]
#     else:
#         graph_dic_train[i] = graph_dic[i-100]
# for i in range(10):
#     graph_dic_test[i] = graph_dic[1]

In [10]:
# graph_dic_test[1].show()

## Feature

In [11]:
# g = graph_dic_train[1]
# g.add_center()
# g.add_edge_between_centers()
# g.add_edge_between_centerAnsBusStation()
# feature = []
# for i in range(40):
#     list_near_stop_i = []
#     feature_i = []
#     for j in range(40,140):
#         if np.linalg.norm( np.array(g.all_pos[i])-np.array(g.all_pos[j]) ) < 15:
#             list_near_stop_i.append( j )
#     for j in list_near_stop_i:
#         if j <=90:
#             feature_i.append(1000)
#         else:
#             feature_i.append(10)
#     feature.append( np.sum(feature_i) )

## 2 Train

In [12]:
import agent
import environment
import runner

In [13]:
# def train():
    
#     #initial parameters
#     lr = 0.001

    
#     #initial agent class and environment class
#     agent_class = agent.Agent(graph_dic_train, lr,'train')
#     env_class = environment.Environment(graph_dic_train)

#     print("Running a single instance simulation...")
#     my_runner = runner.Runner(env_class, agent_class,'train')
#     test_list_min_acc = my_runner.loop(len( graph_dic_train ), 1000)
        
#         #print("Obtained a final reward of {}".format(final_reward))
#     agent_class.save_model()
#     return test_list_min_acc

In [14]:
# result_old = train()

## 3 Test

In [15]:
# import agent
# import environment
# import runner

In [16]:
# def RLtest():
    
#     #initial parameters
#     lr = 0.001

    
#     #initial agent class and environment class
#     agent_class = agent.Agent(graph_dic_test, lr, 'RLtest')
#     env_class = environment.Environment(graph_dic_test)

#     print("Running a single instance simulation...")
#     my_runner = runner.Runner(env_class, agent_class,'RLtest')
#     test_list_min_acc = my_runner.loop(len( graph_dic_test ), 1000)
        
#         #print("Obtained a final reward of {}".format(final_reward))
# #         agent_class.save_model()
#     return test_list_min_acc

In [17]:
# result = RLtest()

In [18]:
# def RDtest(i,number_trees):
    
#     #initial parameters
#     lr = 0.001
    
#     test_set = graph_dic_test[i*number_trees:(i+1)*number_trees]
    
    
#     #initial agent class and environment class
#     agent_class = agent.Agent(test_set, lr, 'RDtest')
#     env_class = environment.Environment(test_set)

#     print("Running a single instance simulation...")
#     my_runner = runner.Runner(env_class, agent_class,'RDtest')
#     test_list_min_acc = my_runner.loop(len( test_set ), 1000)
        
#         #print("Obtained a final reward of {}".format(final_reward))
# #         agent_class.save_model()
#     return test_list_min_acc

In [19]:
import time
import multiprocessing

def RDtest(i):
    
    #initial parameters
    lr = 0.001
    test_set = {}
    for j in range(i*number_trees,(i+1)*number_trees):
        test_set[j-i*number_trees] = graph_dic_test[j]
    
    
    #initial agent class and environment class
    agent_class = agent.Agent(test_set, lr, 'RDtest')
    env_class = environment.Environment(test_set)

    print("Running a single instance simulation...")
    my_runner = runner.Runner(env_class, agent_class,'RDtest')
    test_list_min_acc = my_runner.loop(len( test_set ), 1000)
        
        #print("Obtained a final reward of {}".format(final_reward))
#         agent_class.save_model()
    return test_list_min_acc

In [20]:
time1 = time.time()
nums_list = [i for i in range(number_graphs)]

pool = multiprocessing.Pool(processes=90) 

result = pool.map(RDtest, nums_list)  

pool.close()        
pool.join()        


Running a single instance simulation...Running a single instance simulation...Running a single instance simulation...Running a single instance simulation...Running a single instance simulation...Running a single instance simulation...


Running a single instance simulation...Running a single instance simulation...


Running a single instance simulation...Running a single instance simulation... -> epoch : 0Running a single instance simulation... -> epoch : 0Running a single instance simulation...Running a single instance simulation...Running a single instance simulation...Running a single instance simulation... -> epoch : 0 -> epoch : 0

 -> epoch : 0
 -> epoch : 0


Running a single instance simulation...






 -> epoch : 0
 -> epoch : 0 -> epoch : 0
 -> epoch : 0Running a single instance simulation... -> epoch : 0
 -> epoch : 0 -> epoch : 0Running a single instance simulation... -> epoch : 0 -> epoch : 0




 -> epoch : 0

Running a single instance simulation...

Running a single ins

/home/dwang/Desktop/PHD/Top10_ACC_8lines/graph.py:176: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  return nx.adjacency_matrix(self.g)
/home/dwang/Desktop/PHD/Top10_ACC_8lines/graph.py:176: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  return nx.adjacency_matrix(self.g)
/home/dwang/Desktop/PHD/Top10_ACC_8lines/graph.py:176: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  return nx.adjacency_matrix(self.g)
/home/dwang/Desktop/PHD/Top10_ACC_8lines/graph.py:176: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  return nx.adjacency_matrix(self.g)
/home/dwang/Desktop/PHD/Top10_ACC_8lines/graph.py:176: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  return nx.adjacency_matrix(self.g)
/home/dwang/Desktop/PHD/Top10_

/home/dwang/Desktop/PHD/Top10_ACC_8lines/graph.py:176: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  return nx.adjacency_matrix(self.g)
/home/dwang/Desktop/PHD/Top10_ACC_8lines/graph.py:176: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  return nx.adjacency_matrix(self.g)
/home/dwang/Desktop/PHD/Top10_ACC_8lines/graph.py:176: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  return nx.adjacency_matrix(self.g)
/home/dwang/Desktop/PHD/Top10_ACC_8lines/graph.py:176: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  return nx.adjacency_matrix(self.g)
/home/dwang/Desktop/PHD/Top10_ACC_8lines/graph.py:176: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  return nx.adjacency_matrix(self.g)
/home/dwang/Desktop/PHD/Top10_

 -> epoch : 1
 -> epoch : 1


/home/dwang/Desktop/PHD/Top10_ACC_8lines/graph.py:176: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  return nx.adjacency_matrix(self.g)


...

 -> epoch : 1


/home/dwang/Desktop/PHD/Top10_ACC_8lines/graph.py:176: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  return nx.adjacency_matrix(self.g)


 -> epoch : 1
 -> epoch : 2
 -> epoch : 1


/home/dwang/Desktop/PHD/Top10_ACC_8lines/graph.py:176: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  return nx.adjacency_matrix(self.g)
/home/dwang/Desktop/PHD/Top10_ACC_8lines/graph.py:176: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  return nx.adjacency_matrix(self.g)


 -> epoch : 2
 -> epoch : 2
 -> epoch : 2
 -> epoch : 2
 -> epoch : 2
 -> epoch : 2
 -> epoch : 2
 -> epoch : 2
 -> epoch : 2
 -> epoch : 2
 -> epoch : 2
 -> epoch : 2
 -> epoch : 2
 -> epoch : 2
 -> epoch : 2
 -> epoch : 2
 -> epoch : 2
 -> epoch : 2
 -> epoch : 2
 -> epoch : 2
 -> epoch : 2
 -> epoch : 2
 -> epoch : 2
 -> epoch : 2
 -> epoch : 2
 -> epoch : 2
 -> epoch : 2
 -> epoch : 2
 -> epoch : 2
 -> epoch : 2
 -> epoch : 2
 -> epoch : 2
 -> epoch : 2
 -> epoch : 2
 -> epoch : 2
 -> epoch : 2
 -> epoch : 2
 -> epoch : 2
 -> epoch : 2
 -> epoch : 2
 -> epoch : 2
 -> epoch : 2
 -> epoch : 2
 -> epoch : 2
 -> epoch : 2
 -> epoch : 2
 -> epoch : 3
 -> epoch : 3
 -> epoch : 3
 -> epoch : 3
 -> epoch : 3
 -> epoch : 3
 -> epoch : 3
 -> epoch : 3
 -> epoch : 3
 -> epoch : 3
 -> epoch : 3
 -> epoch : 3
 -> epoch : 3
 -> epoch : 3
 -> epoch : 3
 -> epoch : 3
 -> epoch : 3
 -> epoch : 3
 -> epoch : 3
 -> epoch : 3
 -> epoch : 3
 -> epoch : 3
 -> epoch : 3
 -> epoch : 3
 -> epoch : 3
 -> ep

In [21]:
print("Time used：", time.time()-time1)

Time used： 1892.8699824810028


In [22]:
result_RD = []
for k in range(8):
    result_ = []
    for i in range(0,number_graphs):
        result_ = result_ + result[i][k]
    result_RD.append( result_ ) 

In [23]:
#result_RD = RDtest()

In [24]:
best_tree_index = []

for i in range(number_graphs):
    qq = []
    for j in range(number_trees):
        qq.append( result_RD[1][number_trees*i+j] )
        if len(qq)==number_trees:
            best_tree_index.append( number_trees*i+np.argmax(qq) )

In [25]:
acc = result_RD[3]
Akinson = result_RD[4]
Pietra = result_RD[5]
Theil = result_RD[6]

In [26]:
#np.array( acc[best_tree_index[2]] )

In [27]:
#np.array( initial_acc[2] )

In [28]:
# acc_new = []
# for i in range(len(best_tree_index)):
#     index = best_tree_index[i]
#     acc[index].sort()
#     acc_new.append( acc[index] )

In [29]:
graph_dic = np.load( 'test_set.npy',allow_pickle=True ).item()

graph_dic_test = {}
for i in range(number_graphs):
    graph_dic_test[i] = graph_dic[i]

initial_AWT = []
initial_acc = []
initial_Akinson = []
initial_Pietra = []
initial_Theil = []

for i in range(number_graphs):
    graph_dic_test[i].add_center()
    graph_dic_test[i].add_edge_between_centers()
    graph_dic_test[i].add_edge_between_centerAnsBusStation()
    acc_r = graph_dic_test[i].get_acc()
    initial_acc.append( acc_r[1] )
    initial_Akinson.append( acc_r[2] )
    initial_Pietra.append( acc_r[3] )
    initial_Theil.append( acc_r[4] )
    initial_AWT.append( graph_dic_test[i].avg_frequency() )

In [30]:
dis_map = {}
for i in range(len(best_tree_index)):
    index = best_tree_index[i]
    dis_map[i] = result_RD[7][index]
#node_index = np.array([k for k in range(80)])[ np.reshape(dis_map[i].numpy(),-1)  != 0 ]

In [31]:
sig_no_yes = []
for i in range( number_graphs ):
    sig_no_yes = sig_no_yes + list( 1 - np.reshape(dis_map[0].tolist(),-1) )
sig_no_yes = np.array( sig_no_yes )

In [32]:
sig_no_yes

array([0., 0., 1., ..., 1., 1., 1.])